In [1]:
import os
import csv
import time
import requests
import pandas as pd
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor    # Concurrent execution using threads
gcp_project_id = "artful-tractor-338209"   # Set the Google Cloud Project ID

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# provide dataset name among ai2_arc, gsm8k, lukaemon/mmlu, bbc_hindi_nli(for testing hindi to hinglish)
dataset_name = "bbc_hindi_nli"

In [3]:
if(dataset_name == "ai2_arc"):
    possible_configs = [
    'ARC-Challenge',
    'ARC-Easy'
    ]
elif(dataset_name == "gsm8k"):
    possible_configs = [
    "main",
    "socratic"
    ]
elif(dataset_name == "lukaemon/mmlu"):
    possible_configs = [
    "high_school_european_history",
    "business_ethics",
    "clinical_knowledge",
    "medical_genetics",
    "high_school_us_history",
    "high_school_physics",
    "high_school_world_history",
    "virology",
    "high_school_microeconomics",
    "econometrics",
    "college_computer_science",
    "high_school_biology",
    "abstract_algebra",
    "professional_accounting",
    "philosophy",
    "professional_medicine",
    "nutrition",
    "global_facts",
    "machine_learning",
    "security_studies",
    "public_relations",
    "professional_psychology",
    "prehistory",
    "anatomy",
    "human_sexuality",
    "college_medicine",
    "high_school_government_and_politics",
    "college_chemistry",
    "logical_fallacies",
    "high_school_geography",
    "elementary_mathematics",
    "human_aging",
    "college_mathematics",
    "high_school_psychology",
    "formal_logic",
    "high_school_statistics",
    "international_law",
    "high_school_mathematics",
    "high_school_computer_science",
    "conceptual_physics",
    "miscellaneous",
    "high_school_chemistry",
    "marketing",
    "professional_law",
    "management",
    "college_physics",
    "jurisprudence",
    "world_religions",
    "sociology",
    "us_foreign_policy",
    "high_school_macroeconomics",
    "computer_security",
    "moral_scenarios",
    "moral_disputes",
    "electrical_engineering",
    "astronomy",
    "college_biology",
    ]
elif(dataset_name == "bbc_hindi_nli"):
    possible_configs = [
    "bbc hindi nli"
    ]
    # columns to translate
    columns = ['premise','hypothesis']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = ['label','topic']

In [4]:
dataset = []
for config in possible_configs:
    dataset_slice = load_dataset("bbc_hindi_nli", config,ignore_verifications=True)
    dataset.append(dataset_slice)

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1760: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/15552 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2580 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2592 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/karti/.cache/huggingface/datasets/parquet/bbc hindi nli-c5e28bba7f715fdf/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# dataset
for g in dataset:
    print(g['train'])
    print(g['test'])
    print(g['validation'])

Dataset({
    features: ['premise', 'hypothesis', 'label', 'topic'],
    num_rows: 15552
})
Dataset({
    features: ['premise', 'hypothesis', 'label', 'topic'],
    num_rows: 2592
})
Dataset({
    features: ['premise', 'hypothesis', 'label', 'topic'],
    num_rows: 2580
})


## Get access token for romanization

1. go to link https://developers.google.com/oauthplayground/ 
2. select Cloud Translation API v3, under it select both https://www.googleapis.com/auth/cloud-platform
https://www.googleapis.com/auth/cloud-translation
3. authorize
4. exchange authorization code for tokens and copy the access token.

   #### valid for one hour only

In [7]:
access_token = 'ya29.a0AfB_byBvm1OheATW_139gkhWA7A7eBXG8uZRcMmUyjEXMmBjaebMzVaxlTXXAGJSvSLDS1SxkaGFpViizMtfMlMsB5DHBetdbSiM2gO-H2HdyJmTeaF-OYI2r94yb83k01WigkxB9yErQGnKwD5s4aMM96Ri3uemrhnRaCgYKAUASARASFQHGX2MivUrMpO0a92cuLTn67UitpQ0171'

In [8]:
def romanize( text: str):
    url = 'https://translation.googleapis.com/v3/projects/artful-tractor-338209/locations/global:romanizeText'

    headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    }
    data = {
    'contents': [text],
    'sourceLanguageCode': 'hi',
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        romanized_text = result['romanizations'][0]['romanizedText']
        return f'{romanized_text}'
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [9]:
romanize("कार्तिक आप कैसे हैं?")

'kaartik aap kaise hain?'

In [11]:
%%time

if(dataset_name=='lukaemon/mmlu'):
    os.makedirs("lukaemon_mmlu_files", exist_ok=True)
    os.chdir("lukaemon_mmlu_files")
else:
    os.makedirs(dataset_name + "_files", exist_ok=True)
    os.chdir(dataset_name + "_files")

for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            
            # Use ThreadPoolExecutor for parallel translation
            if __name__ == '__main__':
                result =[]
                with ThreadPoolExecutor(max_workers=16) as exe:
                    # Maps the method with a list of values.
                    # result = list(exe.map(romanize,values[10:12]))
                    
                    batch_size = 1
                    for j in range(0,len(values[:3]),batch_size):
                        j_end = min(j + batch_size, len(values[:3]))
                        res = list(exe.map(romanize,values[j:j_end]))
                        # time.sleep(2)
                        result.extend(res)
                    
            set_list.append(result)

        # Create folders for each configuration
        current_directory = os.getcwd()
        
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True)       
    
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))

        # to add untranslated columns in dataset
        for row in range(len(transposed_data)):
            for col in columns_asis:
                if col=='id':
                        position = 0
                else:
                    position = len(transposed_data[row])
                transposed_data[row].insert(position, dataset[i][set][col][row]) 
        
        path = os.path.join(possible_configs[i], f'{set}.csv')

        # append to previosly created csv file in case full dataset was not converted
        with open(path, 'a', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)

CPU times: total: 1.23 s
Wall time: 10.7 s


In [12]:
df = pd.read_csv(path, header=None)

# Print the DataFrame
df.head()

,0,1,2,3
0,"bam dhamaakon se dahala iraak, darjanon mare",yah bhaarat kee soochana hai|,0,2
1,"bam dhamaakon se dahala iraak, darjanon mare",yah bhaarat kee soochana nahin hai|,1,2
2,"bam dhamaakon se dahala iraak, darjanon mare",yah khabar kee soochana hai|,0,2
